#**Step 1: Install All the Required Packages**

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.7 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 10.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 10.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 58.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 34.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 9.0 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-jc1msv5i/overlay/local/bin
    changing mode of /tmp/pip-build-env-jc1msv5i/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-jc1msv5i/overlay/

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [3]:
from huggingface_hub import hf_hub_download


In [4]:
from llama_cpp import Llama


In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

#**Step 4: Loading the Model**

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**Step 5: Create a Prompt Template**

In [12]:
prompt = '''read the following and identify the city with maximum and minimum rain fall

f1:
[00:00] Good evening, viewers! Welcome to the Weather Update on [Channel Name]. I'm your meteorologist, [Your Name], and I'm here to bring you the latest weather forecast across the diverse landscapes of India. So, grab a cup of chai and let's dive into the atmospheric details!

[00:15] Starting with the northern region, Delhi is experiencing a pleasant evening with a current temperature of 24 degrees Celsius. The skies are mostly clear, and residents can expect a cool night ahead. Moving to Jaipur, temperatures are slightly warmer at 27 degrees Celsius, and the weather remains dry with a gentle breeze.

[00:30] Meanwhile, in the eastern part of the country, Kolkata is enjoying a mild evening with a temperature of 26 degrees Celsius. The humidity is relatively low, making it a comfortable night for everyone in the city. Moving on to Bhubaneswar, the temperature is 25 degrees Celsius, and the skies are clear, promising a tranquil night.

[00:45] Let's shift our focus to the western region. Mumbai is experiencing a milder night compared to the daytime, with the current temperature at 28 degrees Celsius. The coastal breeze is providing some relief from the daytime heat. Moving to Ahmedabad, temperatures are relatively higher at 30 degrees Celsius, and the night is expected to be warm and dry.

[01:00] Now, down south, let's take a look at Chennai. The temperature is currently at 29 degrees Celsius, and the city is experiencing a pleasant evening with a gentle sea breeze. In Bangalore, temperatures are cooler at 22 degrees Celsius, and the night is expected to be comfortable with clear skies.

[01:15] As we move towards the central part of India, Nagpur is experiencing a slightly warmer night with temperatures around 28 degrees Celsius. The atmosphere is dry, and the skies are mostly clear. Bhopal, on the other hand, is enjoying a cooler evening with temperatures at 22 degrees Celsius, and the night is expected to be calm and comfortable.

[01:30] Before we wrap up, a quick look at the overall weather pattern – a high-pressure system is dominating most parts of the country, ensuring stable and clear conditions. However, keep in mind that weather can be unpredictable, so it's always a good idea to stay tuned for any updates.

[01:45] That's all for tonight's weather report. We hope you have a wonderful evening, and don't forget to dress accordingly for the night ahead. Stay tuned to [Channel Name] for more updates on weather, news, and entertainment. Good night and stay weather-wise!

f2:
[00:00] Good evening, viewers! Welcome to your favorite weather update segment. I'm your meteorologist, [Your Name], and I'm here to guide you through the current weather conditions across India. Let's dive into the details!

[00:15] Starting with the northern region, Delhi is experiencing a pleasant evening with a temperature of 25 degrees Celsius. The capital city can expect clear skies throughout the night, so if you're planning to step out, a light jacket should do the trick.

[00:30] Moving towards the vibrant state of Rajasthan, Jaipur is embracing a slightly cooler atmosphere at 22 degrees Celsius. The winds are calm, and the stars are shining brightly. Perfect weather for a leisurely stroll in the Pink City.

[00:45] Now, let's hop over to the western coast. Mumbai, the city that never sleeps, is currently basking in a mild 28 degrees Celsius. The humidity is in check, making it a comfortable night for Mumbaikars.

[01:00] Shifting gears to the southern part of the country, Bengaluru is enjoying a cool 20 degrees Celsius. The IT hub is under the charm of a light drizzle, so don't forget to grab your umbrellas if you're heading out for some late-night coffee.

[01:15] Down south in Chennai, the temperature is hovering around 26 degrees Celsius. A gentle breeze from the Bay of Bengal is bringing relief to the city, creating a pleasant ambiance for the night.

[01:30] Let's swing by the eastern states. Kolkata, the cultural capital, is experiencing a mild 23 degrees Celsius. The skies are clear, and the Howrah Bridge is beautifully illuminated. A great night for those evening addas.

[01:45] As we wrap up our journey across the nation, let's not forget our friends in the Himalayan region. Srinagar is in the midst of a chilly night at 10 degrees Celsius. The snow-capped mountains are gleaming under the moonlight, creating a breathtaking panorama.

[02:00] That concludes our weather report for tonight. Whether you're enjoying a quiet night at home or planning an outing with friends, we hope this update helps you make the most of the weather. Stay tuned for more updates, and have a fantastic evening!

f3:
[00:00] Good evening, viewers! Welcome to the Weather Watch on India Weather Now. I'm your meteorologist, [Your Name], and I'm here to bring you the latest updates on the weather conditions across the country. So, let's dive right into it!

[00:15] Starting with the northern regions, Delhi is currently experiencing a pleasant evening with temperatures around 18 degrees Celsius. In neighboring cities like Jaipur and Lucknow, the mercury is hovering around 20 and 22 degrees Celsius respectively. Make sure to grab a light jacket if you're stepping out.

[00:30] Moving towards the western part of India, Mumbai is enjoying a mild coastal breeze, keeping temperatures at a comfortable 24 degrees Celsius. Pune, on the other hand, is a bit cooler at 19 degrees Celsius. Remember to enjoy the pleasant weather while it lasts!

[00:45] Let's shift our focus to the southern states. Bangalore is experiencing a cool night with temperatures dropping to 17 degrees Celsius. Chennai, with its coastal charm, is maintaining a steady 25 degrees Celsius. A perfect night for a stroll on the beach!

[01:00] Now, heading to the eastern regions, Kolkata is witnessing a mild evening at 22 degrees Celsius. Further northeast, in Guwahati, the temperature is slightly lower, around 19 degrees Celsius. It's a great time to enjoy some hot tea and snacks in these regions.

[01:15] Central India is currently under the influence of a mild weather pattern. Bhopal is reporting a temperature of 21 degrees Celsius, while Nagpur is slightly warmer at 23 degrees Celsius. Residents in these areas can expect a comfortable night ahead.

[01:30] As we look ahead, it's worth noting that the overall weather pattern in the country is stable, with no major disturbances expected in the coming days. However, some regions may experience a slight dip in temperatures, so stay prepared.

[01:45] Before we wrap up, a quick reminder to stay updated with our weather app for real-time information. And that's all for tonight's weather report. Thank you for joining us on India Weather Now. Have a pleasant evening!



f5:
[00:00] Good evening, viewers! Welcome to Weather Watch, your go-to source for the latest weather updates across India. I'm your host, [Your Name], and I'm thrilled to guide you through the current weather conditions in different parts of the country.

[00:15] Let's kick things off by taking a look at the temperatures in some of the major cities. In the northern region, Delhi is experiencing a pleasant 25 degrees Celsius, while up in the mountains, Shimla is a bit cooler at 15 degrees Celsius. Moving towards the western part of India, Mumbai is enjoying a warm 30 degrees Celsius, and Ahmedabad is equally inviting with a temperature of 28 degrees Celsius.

[00:40] Heading eastward, Kolkata is experiencing a comfortable 27 degrees Celsius, and the northeastern city of Guwahati is enjoying a mild 22 degrees Celsius. In the southern part of the country, Chennai is basking in a toasty 32 degrees Celsius, and Bengaluru is a bit cooler at 28 degrees Celsius.

[01:00] Now, let's talk about the overall weather conditions. The monsoon is in full swing across the western coast, bringing relief to states like Kerala and Goa. The central and northern regions are experiencing clear skies and dry weather, making it ideal for outdoor activities.

[01:15] In the eastern part of India, there's a chance of scattered showers in some areas, so keep those umbrellas handy. The northeastern states may experience some foggy conditions in the early morning hours. Down south, coastal areas might witness a slight increase in humidity.

[01:30] If you're planning any travel in the next few days, it's essential to stay informed about the weather conditions. Make sure to check for updates, especially if you're heading towards regions prone to sudden weather changes.

[01:45] Before we wrap up, a quick reminder to our viewers: Stay hydrated, use sunscreen if you're stepping out in the sun, and stay tuned to Weather Watch for the most accurate and up-to-date information.

[02:00] That's all for now. Thank you for joining us on Weather Watch. I'm [Your Name], wishing you a pleasant evening and safe weather ahead. Stay tuned for more updates, and we'll see you next time.
'''
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

Based on the provided weather report, the cities with temperatures less than 25 degrees Celsius are:

1. Shimla (15 degrees Celsius)
2. Guwahati (22 degrees Celsius)
3. Bengaluru (28 degrees Celsius)

Among these cities, Guwahati and Bengaluru have the probability of rain mentioned:

- Guwahati: The report mentions that there's a chance of scattered showers in some areas in the eastern part of India, which includes Guwahati.
- Bengaluru: Although the temperature is below 25 degrees Celsius, there is no explicit mention of rain in the report for Bengaluru. Therefore, we cannot confirm if rain is probable in Bengaluru based solely on this weather report.

So, based on the information provided, Guwahati is the city with a temperature below 25 degrees Celsius and a probability of rain.

#**Step 6: Generating the Response**

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

In [ ]:
print(response)

In [ ]:
print(response["choices"][0]["text"])